In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [48]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [5]:
train_data.targets.size()

torch.Size([60000])

In [6]:
from torch.utils.data import DataLoader

loaders = {

    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),
    
    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)

}

In [7]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x111288950>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x11124bcb0>}

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        self = self.fc2(x)

        return F.softmax(x)

In [9]:
import torch
print(torch.backends.mps.is_available())

# device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')
    

True


In [11]:
import time

device = torch.device('cpu')

model = CNN().to(device)

start_time_cpu = time.time()

for epoch in range(1,6): # 5 epochs
    train(epoch)
    test()

end_time_cpu = time.time() 
total_time_cpu = end_time_cpu - start_time_cpu
#total_time_cpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_4406/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.911497
Train Epoch: 1 [2000/60000 (3%)]	3.911552
Train Epoch: 1 [4000/60000 (7%)]	3.911393
Train Epoch: 1 [6000/60000 (10%)]	3.912340
Train Epoch: 1 [8000/60000 (13%)]	3.911303
Train Epoch: 1 [10000/60000 (17%)]	3.910873
Train Epoch: 1 [12000/60000 (20%)]	3.911616
Train Epoch: 1 [14000/60000 (23%)]	3.911078
Train Epoch: 1 [16000/60000 (27%)]	3.911034
Train Epoch: 1 [18000/60000 (30%)]	3.911508
Train Epoch: 1 [20000/60000 (33%)]	3.911606
Train Epoch: 1 [22000/60000 (37%)]	3.911731
Train Epoch: 1 [24000/60000 (40%)]	3.911243
Train Epoch: 1 [26000/60000 (43%)]	3.911501
Train Epoch: 1 [28000/60000 (47%)]	3.911093
Train Epoch: 1 [30000/60000 (50%)]	3.910631
Train Epoch: 1 [32000/60000 (53%)]	3.910978
Train Epoch: 1 [34000/60000 (57%)]	3.911584
Train Epoch: 1 [36000/60000 (60%)]	3.911054
Train Epoch: 1 [38000/60000 (63%)]	3.910630
Train Epoch: 1 [40000/60000 (67%)]	3.911276
Train Epoch: 1 [42000/60000 (70%)]	3.911237
Train Epoch: 1 [44000/60000 (73%)]	3.911505

In [12]:
device = torch.device('mps') #switching device to GPU

model = CNN().to(device)

start_time_gpu = time.time()

for epoch in range(1,6): # 5 epochs
    train(epoch)
    test()

end_time_gpu = time.time() 
total_time_gpu = end_time_gpu - start_time_gpu
#total_time_gpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_4406/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.911580
Train Epoch: 1 [2000/60000 (3%)]	3.912125
Train Epoch: 1 [4000/60000 (7%)]	3.912163
Train Epoch: 1 [6000/60000 (10%)]	3.912100
Train Epoch: 1 [8000/60000 (13%)]	3.911898
Train Epoch: 1 [10000/60000 (17%)]	3.912363
Train Epoch: 1 [12000/60000 (20%)]	3.912251
Train Epoch: 1 [14000/60000 (23%)]	3.911770
Train Epoch: 1 [16000/60000 (27%)]	3.911974
Train Epoch: 1 [18000/60000 (30%)]	3.911777
Train Epoch: 1 [20000/60000 (33%)]	3.912202
Train Epoch: 1 [22000/60000 (37%)]	3.912111
Train Epoch: 1 [24000/60000 (40%)]	3.912072
Train Epoch: 1 [26000/60000 (43%)]	3.912187
Train Epoch: 1 [28000/60000 (47%)]	3.912158
Train Epoch: 1 [30000/60000 (50%)]	3.912139
Train Epoch: 1 [32000/60000 (53%)]	3.912007
Train Epoch: 1 [34000/60000 (57%)]	3.911742
Train Epoch: 1 [36000/60000 (60%)]	3.911813
Train Epoch: 1 [38000/60000 (63%)]	3.912134
Train Epoch: 1 [40000/60000 (67%)]	3.912203
Train Epoch: 1 [42000/60000 (70%)]	3.911808
Train Epoch: 1 [44000/60000 (73%)]	3.911544

In [14]:
print('Stats for 5 epochs:')
print(f"Time training for CPU is: {total_time_cpu:.2f} seconds.")
print(f"Time training for GPU is: {total_time_gpu:.2f} seconds.\n")

if total_time_cpu > total_time_gpu:
    print(f"The GPU outpreformed the CPU for: {total_time_cpu-total_time_gpu:.2f} seconds.")
    print(f"The GPU was: {total_time_cpu/total_time_gpu:2f} times faster.")
    print(f"The GPU was: {(total_time_cpu - total_time_gpu) / total_time_cpu * 100:2f} % faster.")
else:
    print(f"The CPU outpreformed the CPU for: {total_time_gpu-total_time_cpu:.2f} seconds. or {total_time_cpu/total_time_gpu*100:2f}%.")
    print(f"The CPU was: {total_time_gpu/total_time_cpu:2f} times faster.")
    print(f"The CPU was: {(total_time_gpu - total_time_cpu) / total_time_gpu * 100:2f} % faster.")

Stats for 5 epochs:
Time training for CPU is: 35.92 seconds.
Time training for GPU is: 24.58 seconds.

The GPU outpreformed the CPU for: 11.34 seconds.
The GPU was: 1.461479 times faster.
The GPU was: 31.576156 % faster.


In [ ]:
device = torch.device('cpu')

model = CNN().to(device)

start_time_cpu = time.time()

for epoch in range(1,11): # 10 epochs
    train(epoch)
    test()

end_time_cpu = time.time() 
total_time_cpu = end_time_cpu - start_time_cpu

In [ ]:
device = torch.device('mps') #switching device to GPU

model = CNN().to(device)

start_time_gpu = time.time()

for epoch in range(1,11): # 10 epochs
    train(epoch)
    test()

end_time_gpu = time.time() 
total_time_gpu = end_time_gpu - start_time_gpu

Time training for CPU is: 67.75 seconds.
Time training for GPU is: 48.18 seconds.

The GPU outpreformed the CPU for: 19.57 seconds.
The GPU was: 1.406161 times faster.
The GPU was: 28.884371 % faster.
